This notebook shows how to use `lightgbm.dask` to train a LightGBM model on data stored as a [Dask DataFrame](https://docs.dask.org/en/latest/dataframe.html) or [Dask Array](https://docs.dask.org/en/latest/array.html).

It uses `FargateCluster` from [`dask-cloudprovider`](https://github.com/dask/dask-cloudprovider) to create a distributed cluster.

<hr>

## Set up a cluster on AWS Fargate

In [ ]:
import json

with open("ecr-details.json", "r") as f:
    ecr_details = json.loads(f.read())

CONTAINER_IMAGE = ecr_details["repository"]["repositoryUri"] + ":1"
print(f"scheduler and worker image: {CONTAINER_IMAGE}")

Before proceeding, set up your AWS credentials. If you're unsure how to do this, see [the AWS docs](https://boto3.amazonaws.com/v1/documentation/api/latest/guide/credentials.html).

In [ ]:
import os

os.environ["AWS_DEFAULT_REGION"] = "us-west-2"

Create a cluster with 3 workers. See https://cloudprovider.dask.org/en/latest/aws.html#dask_cloudprovider.aws.FargateCluster for more options.

In [ ]:
from dask_cloudprovider.aws import FargateCluster
from dask.distributed import Client

n_workers = 3
cluster = FargateCluster(
    image=CONTAINER_IMAGE,
    worker_cpu=512,
    worker_mem=4096,
    n_workers=n_workers,
    fargate_use_private_ip=False,
    scheduler_timeout="40 minutes",
    find_address_timeout=60 * 10,
)
client = Client(cluster)
client.wait_for_workers(n_workers)

In [ ]:
print(f"View the dashboard: {cluster.dashboard_link}")

Click the link above to view a diagnostic dashboard while you run the training code below.

<hr>

## Train a model

In [ ]:
import dask.array as da
from dask.distributed import wait
from lightgbm.dask import DaskLGBMRegressor

In [ ]:
for i in range(10):
    print(f"attempt {i}")
    
    client.restart()

    num_rows = 1e6
    num_features = 1e2
    num_partitions = 10
    rows_per_chunk = num_rows / num_partitions

    data = da.random.random((num_rows, num_features), (rows_per_chunk, num_features))

    labels = da.random.random((num_rows, 1), (rows_per_chunk, 1))

    data = data.persist()
    labels = labels.persist()
    _ = wait(data)
    _ = wait(labels)

    dask_reg = DaskLGBMRegressor(
        silent=False,
        max_depth=5,
        random_state=708,
        objective="regression_l2",
        learning_rate=0.1,
        tree_learner="data",
        n_estimators=10,
        min_child_samples=1,
        n_jobs=-1,
        local_listen_port=12400,
    )

    dask_reg.fit(
        client=client,
        X=data,
        y=labels,
    )

The model produced by this training run is an instance of `DaskLGBMRegressor`. To get a regular non-Dask model (which can be pickled and saved), run `.to_local()`.

In [ ]:
local_model = dask_reg.to_local()
type(local_model)

You can visualize this model by looking at a data frame representation of it. You can also check that the model really used inputs from all workers.

In [ ]:
dask_reg.booster_.trees_to_dataframe().iloc[0,]["count"] == 1e6